# 2. Download text for the relevant books
This makes a list of all the English language text download links on Gutenberg, then merges it with the list of English-language fiction produced in stage 1.

Follows Project Gutenberg's [robot readme rules](http://www.gutenberg.org/policy/robot_access.html).


In [ ]:
import requests
import pickle
import zipfile
import io
import sys
import pandas as pd

from bs4 import BeautifulSoup
from pathlib import Path
from multiprocessing.dummy import Pool 

In [ ]:
# Starting page for getting download links from Project Gutenberg
url = 'http://www.gutenberg.org/robot/harvest?filetypes[]=txt&langs[]=en'

links = []
while True:
    # Get the page content and parse with BeautifulSoup
    page = requests.get(url, timeout=15)
    content = BeautifulSoup(page.text, "lxml")
    
    # Find all the book urls on the page
    urls = [e.get('href') for e in content.select("body > p > a[href^='http://aleph.gutenberg.org/']")]
    if urls:
        links.extend(urls)
    
    # Check if there's a 'Next page' button - if there is, visit it and repeat
    next_page = content.find('a', string='Next Page')
    if next_page:
        url = "http://www.gutenberg.org/robot/" + next_page.get("href")
    else:
        break

print(len(links))

In [ ]:
# Load the dataframe of books from the previous notebook - this has catalogue numbers
data_dir = Path.cwd().parent / "data"
books = pd.read_pickle(data_dir / "english_fiction.pkl")

# Work out catalogue numbers from the new list of download urls and make it a dataframe
link_ids = [(url[url.rfind("/") + 1:-4], url) for url in links]
link_ids = pd.DataFrame(link_ids, columns=["catalogue_number", "download"])

# Merge the two dataframes
books = books.merge(link_ids, how="inner", on="catalogue_number")

In [ ]:
# Open a multiprocessing pool and download all the books from the merged list
pool = Pool(4) 
book_links = books["download"].tolist()
storage = data_dir / "gutenberg_text"

def get_book(url):
    try:
        response = requests.get(url)
        file = zipfile.ZipFile(io.BytesIO(response.content))
        file.extractall(storage)
    except:
        problems.append([url, sys.exc_info()[0]])    
        
pool.map(get_book, book_links)

pool.close()
pool.join()